In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Revisor y Detective de Código para Detector_errores.ipynb
Autor: Sistema de Revisión Automática
Fecha: 2024
Descripción: Analiza, detecta errores y optimiza el código del archivo Detector_errores.ipynb
"""

# Celda 1: Importaciones necesarias
import os
import sys
import json
import ast
import traceback
import re
from datetime import datetime
from pathlib import Path
import nbformat
from nbformat import read, write, v4
import pandas as pd
from typing import Dict, List, Tuple, Any, Optional
import logging
import autopep8
import pycodestyle
import pylint.lint
from io import StringIO
import subprocess
import difflib
import yaml
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Celda 2: Configuración del sistema
class ConfiguracionRevisor:
    """Configuración principal del sistema de revisión"""
    
    def __init__(self):
        self.ruta_base = Path(r"C:\Users\Alicia_Piero\Documents\Repo_AIEP\MIP_QUILLOTA\Proyecto_METGO_3D")
        self.archivo_objetivo = "Detector_errores.ipynb"
        self.ruta_completa = self.ruta_base / self.archivo_objetivo
        
        # Crear estructura de directorios
        self.dir_reportes = self.ruta_base / "reportes_revision"
        self.dir_corregidos = self.ruta_base / "notebooks_corregidos"
        self.dir_logs = self.ruta_base / "logs"
        self.dir_backups = self.ruta_base / "backups"
        
        for directorio in [self.dir_reportes, self.dir_corregidos, self.dir_logs, self.dir_backups]:
            directorio.mkdir(exist_ok=True)
        
        # Configurar logging
        self.configurar_logging()
        
        # Estadísticas
        self.estadisticas = {
            'total_celdas': 0,
            'celdas_con_errores': 0,
            'errores_sintaxis': 0,
            'errores_importacion': 0,
            'errores_logica': 0,
            'advertencias': 0,
            'mejoras_aplicadas': 0,
            'tiempo_ejecucion': 0
        }
    
    def configurar_logging(self):
        """Configura el sistema de logging"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        log_file = self.dir_logs / f"revision_{timestamp}.log"
        
        # Configurar formato de logging
        log_format = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        
        # Handler para archivo
        file_handler = logging.FileHandler(log_file, encoding='utf-8')
        file_handler.setFormatter(log_format)
        
        # Handler para consola
        console_handler = logging.StreamHandler(sys.stdout)
        console_handler.setFormatter(log_format)
        
        # Configurar logger
        self.logger = logging.getLogger('RevisorCodigo')
        self.logger.setLevel(logging.INFO)
        self.logger.addHandler(file_handler)
        self.logger.addHandler(console_handler)
        
        self.logger.info("="*80)
        self.logger.info(f"Iniciando revisión de: {self.archivo_objetivo}")
        self.logger.info(f"Ruta completa: {self.ruta_completa}")
        self.logger.info("="*80)

# Celda 3: Analizador de código Python
class AnalizadorCodigo:
    """Analiza el código Python en busca de errores y problemas"""
    
    def __init__(self, config: ConfiguracionRevisor):
        self.config = config
        self.logger = config.logger
        self.resultados = []
        
    def analizar_sintaxis(self, codigo: str, num_celda: int) -> Dict:
        """Analiza la sintaxis del código"""
        resultado = {
            'celda': num_celda,
            'tipo_analisis': 'sintaxis',
            'errores': [],
            'advertencias': [],
            'codigo_original': codigo,
            'codigo_corregido': None,
            'valido': True
        }
        
        try:
            # Intentar parsear el código
            tree = ast.parse(codigo)
            
            # Analizar el árbol AST
            visitor = AnalizadorAST()
            visitor.visit(tree)
            
            # Agregar información del análisis
            resultado['ast_info'] = {
                'num_funciones': len(visitor.funciones),
                'num_clases': len(visitor.clases),
                'num_imports': len(visitor.imports),
                'variables_globales': len(visitor.variables_globales),
                'complejidad_ciclomatica': visitor.complejidad
            }
            
        except SyntaxError as e:
            resultado['valido'] = False
            resultado['errores'].append({
                'tipo': 'SyntaxError',
                'linea': e.lineno,
                'columna': e.offset,
                'mensaje': str(e.msg),
                'texto': e.text,
                'severidad': 'CRITICO',
                'sugerencia': self._sugerir_correccion_sintaxis(e, codigo)
            })
            self.config.estadisticas['errores_sintaxis'] += 1
            
        except Exception as e:
            resultado['valido'] = False
            resultado['errores'].append({
                'tipo': 'Error General',
                'mensaje': str(e),
                'severidad': 'ALTO'
            })
            
        return resultado
    
    def _sugerir_correccion_sintaxis(self, error: SyntaxError, codigo: str) -> str:
        """Sugiere correcciones para errores de sintaxis comunes"""
        sugerencias = []
        
        # Patrones comunes de errores
        if "invalid syntax" in str(error):
            if ":" in str(error):
                sugerencias.append("Verificar que todos los bloques (if, for, while, def, class) terminen con ':'")
            if "(" in str(error) or ")" in str(error):
                sugerencias.append("Verificar paréntesis balanceados")
                
        elif "unexpected indent" in str(error):
            sugerencias.append("Revisar la indentación (usar 4 espacios consistentemente)")
            
        elif "EOL while scanning" in str(error):
            sugerencias.append("Verificar comillas no cerradas en strings")
            
        return " | ".join(sugerencias) if sugerencias else "Revisar sintaxis en la línea indicada"
    
    def analizar_imports(self, codigo: str, num_celda: int) -> Dict:
        """Analiza los imports y detecta problemas"""
        resultado = {
            'celda': num_celda,
            'tipo_analisis': 'imports',
            'errores': [],
            'advertencias': [],
            'imports_encontrados': [],
            'imports_no_usados': [],
            'imports_faltantes': []
        }
        
        try:
            tree = ast.parse(codigo)
            
            # Extraer todos los imports
            imports = []
            for node in ast.walk(tree):
                if isinstance(node, ast.Import):
                    for alias in node.names:
                        imports.append({
                            'modulo': alias.name,
                            'alias': alias.asname,
                            'linea': node.lineno
                        })
                elif isinstance(node, ast.ImportFrom):
                    modulo = node.module or ''
                    for alias in node.names:
                        imports.append({
                            'modulo': f"{modulo}.{alias.name}" if modulo else alias.name,
                            'alias': alias.asname,
                            'desde': modulo,
                            'linea': node.lineno
                        })
            
            resultado['imports_encontrados'] = imports
            
            # Verificar imports no usados
            codigo_sin_imports = self._remover_imports(codigo)
            for imp in imports:
                nombre_buscar = imp['alias'] or imp['modulo'].split('.')[-1]
                if nombre_buscar not in codigo_sin_imports:
                    resultado['imports_no_usados'].append(imp)
                    resultado['advertencias'].append({
                        'tipo': 'Import no usado',
                        'modulo': imp['modulo'],
                        'linea': imp['linea'],
                        'severidad': 'BAJO'
                    })
            
            # Detectar posibles imports faltantes
            patrones_modulos = {
                r'\bpd\.': 'pandas as pd',
                r'\bnp\.': 'numpy as np',
                r'\bplt\.': 'matplotlib.pyplot as plt',
                r'\bsns\.': 'seaborn as sns',
                r'\btf\.': 'tensorflow as tf',
                r'\btorch\.': 'torch',
                r'\bos\.': 'os',
                r'\bsys\.': 'sys',
                r'\bjson\.': 'json',
                r'\bre\.': 're',
                r'\bdatetime\.': 'datetime',
                r'\bPath\(': 'from pathlib import Path',
                r'\blogging\.': 'logging',
                r'\bast\.': 'ast',
                r'\bnbformat\.': 'nbformat',
                r'\bdefaultdict\(': 'from collections import defaultdict',
                r'\bCounter\(': 'from collections import Counter',
                r'\bwarnings\.': 'warnings'
            }
            
            imports_actuales = {imp['modulo'] for imp in imports}
            
            for patron, modulo_sugerido in patrones_modulos.items():
                if re.search(patron, codigo) and modulo_sugerido not in str(imports_actuales):
                    resultado['imports_faltantes'].append({
                        'modulo': modulo_sugerido,
                        'patron_detectado': patron,
                        'severidad': 'MEDIO'
                    })
                    resultado['errores'].append({
                        'tipo': 'Import faltante',
                        'mensaje': f"Se detectó uso de '{patron}' pero no se encontró el import: {modulo_sugerido}",
                        'severidad': 'MEDIO',
                        'sugerencia': f"Agregar: import {modulo_sugerido}"
                    })
            
        except Exception as e:
            resultado['errores'].append({
                'tipo': 'Error al analizar imports',
                'mensaje': str(e),
                'severidad': 'ALTO'
            })
            
        return resultado
    
    def _remover_imports(self, codigo: str) -> str:
        """Remueve las líneas de import del código"""
        lineas = codigo.split('\n')
        lineas_sin_imports = []
        
        for linea in lineas:
            if not (linea.strip().startswith('import ') or 
                   linea.strip().startswith('from ') or
                   linea.strip().startswith('#')):
                lineas_sin_imports.append(linea)
                
        return '\n'.join(lineas_sin_imports)
    
    def analizar_estilo(self, codigo: str, num_celda: int) -> Dict:
        """Analiza el estilo del código usando PEP8"""
        resultado = {
            'celda': num_celda,
            'tipo_analisis': 'estilo',
            'errores': [],
            'advertencias': [],
            'codigo_formateado': None
        }
        
        try:
            # Aplicar autopep8 para obtener código formateado
            codigo_formateado = autopep8.fix_code(codigo, options={'aggressive': 1})
            resultado['codigo_formateado'] = codigo_formateado
            
            # Analizar con pycodestyle
            style_guide = pycodestyle.StyleGuide(quiet=True)
            
            # Crear archivo temporal para análisis
            import tempfile
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as tmp:
                tmp.write(codigo)
                tmp_name = tmp.name
            
            # Ejecutar análisis
            result = style_guide.check_files([tmp_name])
            
            # Procesar resultados
            if result.total_errors > 0:
                for error_code, line_number, column, text, physical_line in result._deferred_print:
                    severidad = 'BAJO' if error_code.startswith('W') else 'MEDIO'
                    resultado['advertencias' if severidad == 'BAJO' else 'errores'].append({
                        'codigo': error_code,
                        'linea': line_number,
                        'columna': column,
                        'mensaje': text,
                        'linea_codigo': physical_line,
                        'severidad': severidad
                    })
            
            # Limpiar archivo temporal
            os.unlink(tmp_name)
            
        except Exception as e:
            resultado['errores'].append({
                'tipo': 'Error al analizar estilo',
                'mensaje': str(e),
                'severidad': 'BAJO'
            })
            
        return resultado
    
    def analizar_logica(self, codigo: str, num_celda: int) -> Dict:
        """Analiza la lógica del código y detecta problemas potenciales"""
        resultado = {
            'celda': num_celda,
            'tipo_analisis': 'logica',
            'errores': [],
            'advertencias': [],
            'mejoras_sugeridas': []
        }
        
        try:
            tree = ast.parse(codigo)
            
            # Detectar variables no definidas
            visitor = DetectorVariablesNoDefinidas()
            visitor.visit(tree)
            
            for var in visitor.variables_no_definidas:
                resultado['errores'].append({
                    'tipo': 'Variable no definida',
                    'variable': var,
                    'severidad': 'ALTO',
                    'sugerencia': f"Asegurarse de que '{var}' esté definida antes de usarla"
                })
            
            # Detectar código muerto
            detector_codigo_muerto = DetectorCodigoMuerto()
            detector_codigo_muerto.visit(tree)
            
            for codigo_muerto in detector_codigo_muerto.codigo_muerto:
                resultado['advertencias'].append({
                    'tipo': 'Código muerto detectado',
                    'linea': codigo_muerto['linea'],
                    'descripcion': codigo_muerto['descripcion'],
                    'severidad': 'BAJO'
                })
            
            # Detectar bucles infinitos potenciales
            detector_bucles = DetectorBuclesInfinitos()
            detector_bucles.visit(tree)
            
            for bucle in detector_bucles.bucles_sospechosos:
                resultado['advertencias'].append({
                    'tipo': 'Posible bucle infinito',
                    'linea': bucle['linea'],
                    'descripcion': bucle['descripcion'],
                    'severidad': 'ALTO',
                    'sugerencia': 'Verificar condición de salida del bucle'
                })
            
            # Detectar manejo de excepciones inadecuado
            detector_excepciones = DetectorExcepciones()
            detector_excepciones.visit(tree)
            
            for excepcion in detector_excepciones.problemas:
                resultado['advertencias'].append({
                    'tipo': 'Manejo de excepciones',
                    'problema': excepcion['problema'],
                    'linea': excepcion['linea'],
                    'severidad': 'MEDIO',
                    'sugerencia': excepcion['sugerencia']
                })
            
            # Analizar complejidad
            analizador_complejidad = AnalizadorComplejidad()
            analizador_complejidad.visit(tree)
            
            if analizador_complejidad.max_complejidad > 10:
                resultado['advertencias'].append({
                    'tipo': 'Complejidad alta',
                    'valor': analizador_complejidad.max_complejidad,
                    'funcion': analizador_complejidad.funcion_compleja,
                    'severidad': 'MEDIO',
                    'sugerencia': 'Considerar refactorizar en funciones más pequeñas'
                })
                
        except Exception as e:
            resultado['errores'].append({
                'tipo': 'Error al analizar lógica',
                'mensaje': str(e),
                'severidad': 'MEDIO'
            })
            
        return resultado

# Celda 4: Clases auxiliares para análisis AST
class AnalizadorAST(ast.NodeVisitor):
    """Visitador AST para extraer información del código"""
    
    def __init__(self):
        self.funciones = []
        self.clases = []
        self.imports = []
        self.variables_globales = []
        self.complejidad = 0
        
    def visit_FunctionDef(self, node):
        self.funciones.append({
            'nombre': node.name,
            'linea': node.lineno,
            'argumentos': len(node.args.args),
            'decoradores': [d.id for d in node.decorator_list if hasattr(d, 'id')],
            'docstring': ast.get_docstring(node)
        })
        self.generic_visit(node)
        
    def visit_ClassDef(self, node):
        self.clases.append({
            'nombre': node.name,
            'linea': node.lineno,
            'bases': [base.id for base in node.bases if hasattr(base, 'id')],
            'docstring': ast.get_docstring(node)
        })
        self.generic_visit(node)
        
    def visit_Import(self, node):
        for alias in node.names:
            self.imports.append({
                'modulo': alias.name,
                'alias': alias.asname,
                'linea': node.lineno
            })
        self.generic_visit(node)
        
    def visit_ImportFrom(self, node):
        for alias in node.names:
            self.imports.append({
                'modulo': f"{node.module}.{alias.name}" if node.module else alias.name,
                'alias': alias.asname,
                'desde': node.module,
                'linea': node.lineno
            })
        self.generic_visit(node)
        
    def visit_Assign(self, node):
        # Detectar asignaciones a nivel global
        for target in node.targets:
            if isinstance(target, ast.Name) and isinstance(target.ctx, ast.Store):
                self.variables_globales.append({
                    'nombre': target.id,
                    'linea': node.lineno
                })
        self.generic_visit(node)

class DetectorVariablesNoDefinidas(ast.NodeVisitor):
    """Detecta variables usadas pero no definidas"""
    
    def __init__(self):
        self.variables_definidas = set()
        self.variables_usadas = set()
        self.variables_no_definidas = set()
        self.ambito_actual = [set()]  # Pila de ámbitos
        
    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Store):
            self.ambito_actual[-1].add(node.id)
        elif isinstance(node.ctx, ast.Load):
            # Verificar si la variable está definida en algún ámbito
            definida = False
            for ambito in self.ambito_actual:
                if node.id in ambito:
                    definida = True
                    break
            
            if not definida and node.id not in __builtins__:
                self.variables_no_definidas.add(node.id)
                
        self.generic_visit(node)
        
    def visit_FunctionDef(self, node):
        # Crear nuevo ámbito para la función
        nuevo_ambito = set()
        
        # Agregar parámetros al ámbito
        for arg in node.args.args:
            nuevo_ambito.add(arg.arg)
            
        self.ambito_actual.append(nuevo_ambito)
        self.generic_visit(node)
        self.ambito_actual.pop()
        
    def visit_ClassDef(self, node):
        # Crear nuevo ámbito para la clase
        self.ambito_actual.append(set())
        self.generic_visit(node)
        self.ambito_actual.pop()

class DetectorCodigoMuerto(ast.NodeVisitor):
    """Detecta código que nunca se ejecutará"""
    
    def __init__(self):
        self.codigo_muerto = []
        self.en_funcion = False
        self.despues_return = False
        
    def visit_FunctionDef(self, node):
        self.en_funcion = True
        self.despues_return = False
        self.generic_visit(node)
        self.en_funcion = False
        
    def visit_Return(self, node):
        self.despues_return = True
        self.generic_visit(node)
        
    def visit_If(self, node):
        # Detectar if True/False
        if isinstance(node.test, ast.Constant):
            if node.test.value is False:
                self.codigo_muerto.append({
                    'linea': node.lineno,
                    'descripcion': 'Bloque if con condición siempre False'
                })
            elif node.test.value is True and node.orelse:
                self.codigo_muerto.append({
                    'linea': node.orelse[0].lineno if node.orelse else node.lineno,
                    'descripcion': 'Bloque else nunca se ejecutará (if siempre True)'
                })
        
        # Verificar código después de return
        if self.despues_return and self.en_funcion:
            self.codigo_muerto.append({
                'linea': node.lineno,
                'descripcion': 'Código después de return no se ejecutará'
            })
            
        self.generic_visit(node)

class DetectorBuclesInfinitos(ast.NodeVisitor):
    """Detecta posibles bucles infinitos"""
    
    def __init__(self):
        self.bucles_sospechosos = []
        
    def visit_While(self, node):
        # Detectar while True sin break
        if isinstance(node.test, ast.Constant) and node.test.value is True:
            tiene_break = self._buscar_break(node)
            if not tiene_break:
                self.bucles_sospechosos.append({
                    'linea': node.lineno,
                    'descripcion': 'while True sin break evidente'
                })
                
        self.generic_visit(node)
        
    def _buscar_break(self, node):
        """Busca instrucciones break en el nodo"""
        for child in ast.walk(node):
            if isinstance(child, ast.Break):
                return True
        return False

class DetectorExcepciones(ast.NodeVisitor):
    """Detecta problemas en el manejo de excepciones"""
    
    def __init__(self):
        self.problemas = []
        
    def visit_ExceptHandler(self, node):
        # Detectar except vacío o demasiado genérico
        if node.type is None:
            self.problemas.append({
                'problema': 'Except sin tipo específico (bare except)',
                'linea': node.lineno,
                'sugerencia': 'Especificar el tipo de excepción a capturar'
            })
        elif hasattr(node.type, 'id') and node.type.id == 'Exception':
            self.problemas.append({
                'problema': 'Captura de Exception muy genérica',
                'linea': node.lineno,
                'sugerencia': 'Considerar capturar excepciones más específicas'
            })
            
        # Detectar pass en except
        if len(node.body) == 1 and isinstance(node.body[0], ast.Pass):
            self.problemas.append({
                'problema': 'Except con solo pass (silencia errores)',
                'linea': node.lineno,
                'sugerencia': 'Al menos registrar o manejar la excepción'
            })
            
        self.generic_visit(node)

class AnalizadorComplejidad(ast.NodeVisitor):
    """Analiza la complejidad ciclomática del código"""
    
    def __init__(self):
        self.complejidad_actual = 0
        self.max_complejidad = 0
        self.funcion_compleja = None
        self.funciones_complejidad = {}
        
    def visit_FunctionDef(self, node):
        # Guardar estado anterior
        complejidad_anterior = self.complejidad_actual
        self.complejidad_actual = 1  # Complejidad base
        
        # Visitar el cuerpo de la función
        self.generic_visit(node)
        
        # Guardar complejidad de esta función
        self.funciones_complejidad[node.name] = self.complejidad_actual
        
        # Actualizar máximo si es necesario
        if self.complejidad_actual > self.max_complejidad:
            self.max_complejidad = self.complejidad_actual
            self.funcion_compleja = node.name
            
        # Restaurar complejidad anterior
        self.complejidad_actual = complejidad_anterior
        
    def visit_If(self, node):
        self.complejidad_actual += 1
        self.generic_visit(node)
        
    def visit_While(self, node):
        self.complejidad_actual += 1
        self.generic_visit(node)
        
    def visit_For(self, node):
        self.complejidad_actual += 1
        self.generic_visit(node)
        
    def visit_ExceptHandler(self, node):
        self.complejidad_actual += 1
        self.generic_visit(node)
        
    def visit_With(self, node):
        self.complejidad_actual += 1
        self.generic_visit(node)
        
    def visit_Assert(self, node):
        self.complejidad_actual += 1
        self.generic_visit(node)
        
    def visit_BoolOp(self, node):
        # Cada operador booleano adicional aumenta la complejidad
        self.complejidad_actual += len(node.values) - 1
        self.generic_visit(node)

# Celda 5: Revisor principal del notebook
class RevisorNotebook:
    """Clase principal para revisar notebooks de Jupyter"""
    
    def __init__(self, config: ConfiguracionRevisor):
        self.config = config
        self.logger = config.logger
        self.analizador = AnalizadorCodigo(config)
        self.resultados_totales = []
        self.notebook_original = None
        self.notebook_corregido = None
        
    def cargar_notebook(self) -> bool:
        """Carga el notebook a revisar"""
        try:
            self.logger.info(f"Cargando notebook: {self.config.ruta_completa}")
            
            if not self.config.ruta_completa.exists():
                self.logger.error(f"El archivo no existe: {self.config.ruta_completa}")
                return False
                
            with open(self.config.ruta_completa, 'r', encoding='utf-8') as f:
                self.notebook_original = nbformat.read(f, as_version=4)
                
            self.logger.info(f"Notebook cargado exitosamente. Total de celdas: {len(self.notebook_original.cells)}")
            self.config.estadisticas['total_celdas'] = len(self.notebook_original.cells)
            
            # Crear backup
            self._crear_backup()
            
            return True
            
        except Exception as e:
            self.logger.error(f"Error al cargar notebook: {str(e)}")
            self.logger.error(traceback.format_exc())
            return False
    
    def _crear_backup(self):
        """Crea un backup del notebook original"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_path = self.config.dir_backups / f"{self.config.archivo_objetivo}.backup_{timestamp}"
        
        with open(backup_path, 'w', encoding='utf-8') as f:
            nbformat.write(self.notebook_original, f)
            
        self.logger.info(f"Backup creado: {backup_path}")
    
    def revisar_notebook(self):
        """Revisa todo el notebook celda por celda"""
        self.logger.info("="*80)
        self.logger.info("INICIANDO REVISIÓN COMPLETA DEL NOTEBOOK")
        self.logger.info("="*80)
        
        inicio = datetime.now()
        
        # Crear copia para correcciones
        self.notebook_corregido = nbformat.v4.new_notebook()
        self.notebook_corregido.metadata = self.notebook_original.metadata.copy()
        
        for idx, celda in enumerate(self.notebook_original.cells):
            self.logger.info(f"\nRevisando celda {idx + 1}/{len(self.notebook_original.cells)}")
            
            if celda.cell_type == 'code':
                resultado_celda = self._revisar_celda_codigo(celda, idx)
                self.resultados_totales.append(resultado_celda)
                
                # Crear celda corregida
                celda_corregida = self._crear_celda_corregida(celda, resultado_celda)
                self.notebook_corregido.cells.append(celda_corregida)
                
            elif celda.cell_type == 'markdown':
                # Revisar markdown
                resultado_celda = self._revisar_celda_markdown(celda, idx)
                self.resultados_totales.append(resultado_celda)
                
                # Copiar celda markdown (posiblemente corregida)
                celda_corregida = nbformat.v4.new_markdown_cell(
                    source=resultado_celda.get('contenido_corregido', celda.source)
                )
                self.notebook_corregido.cells.append(celda_corregida)
                
            else:
                # Copiar otros tipos de celdas sin cambios
                self.notebook_corregido.cells.append(celda)
        
        fin = datetime.now()
        self.config.estadisticas['tiempo_ejecucion'] = (fin - inicio).total_seconds()
        
        self.logger.info("\n" + "="*80)
        self.logger.info("REVISIÓN COMPLETADA")
        self.logger.info("="*80)
        
    def _revisar_celda_codigo(self, celda, idx: int) -> Dict:
        """Revisa una celda de código"""
        codigo = celda.source
        resultado = {
            'indice': idx,
            'tipo_celda': 'code',
            'codigo_original': codigo,
            'errores': [],
            'advertencias': [],
            'correcciones': [],
            'metricas': {}
        }
        
        # Si la celda está vacía
        if not codigo.strip():
            resultado['advertencias'].append({
                'tipo': 'Celda vacía',
                'mensaje': 'La celda no contiene código',
                'severidad': 'BAJO'
            })
            return resultado
        
        # Análisis de sintaxis
        self.logger.info("  - Analizando sintaxis...")
        analisis_sintaxis = self.analizador.analizar_sintaxis(codigo, idx)
        resultado['errores'].extend(analisis_sintaxis.get('errores', []))
        resultado['advertencias'].extend(analisis_sintaxis.get('advertencias', []))
        
        # Si hay errores de sintaxis, intentar corregir
        if not analisis_sintaxis['valido']:
            codigo_corregido = self._intentar_corregir_sintaxis(codigo)
            if codigo_corregido != codigo:
                resultado['correcciones'].append({
                    'tipo': 'Corrección de sintaxis',
                    'descripcion': 'Se aplicaron correcciones automáticas de sintaxis'
                })
                codigo = codigo_corregido
        
        # Análisis de imports
        self.logger.info("  - Analizando imports...")
        analisis_imports = self.analizador.analizar_imports(codigo, idx)
        resultado['errores'].extend(analisis_imports.get('errores', []))
        resultado['advertencias'].extend(analisis_imports.get('advertencias', []))
        resultado['metricas']['imports'] = {
            'total': len(analisis_imports.get('imports_encontrados', [])),
            'no_usados': len(analisis_imports.get('imports_no_usados', [])),
            'faltantes': len(analisis_imports.get('imports_faltantes', []))
        }
        
        # Análisis de estilo
        self.logger.info("  - Analizando estilo PEP8...")
        analisis_estilo = self.analizador.analizar_estilo(codigo, idx)
        resultado['advertencias'].extend(analisis_estilo.get('advertencias', []))
        
        # Aplicar formato si hay código formateado
        if analisis_estilo.get('codigo_formateado'):
            codigo_formateado = analisis_estilo['codigo_formateado']
            if codigo_formateado != codigo:
                resultado['correcciones'].append({
                    'tipo': 'Formato PEP8',
                    'descripcion': 'Se aplicó formato según PEP8'
                })
                codigo = codigo_formateado
        
        # Análisis de lógica
        self.logger.info("  - Analizando lógica...")
        analisis_logica = self.analizador.analizar_logica(codigo, idx)
        resultado['errores'].extend(analisis_logica.get('errores', []))
        resultado['advertencias'].extend(analisis_logica.get('advertencias', []))
        resultado['mejoras_sugeridas'] = analisis_logica.get('mejoras_sugeridas', [])
        
        # Guardar código final (con correcciones aplicadas)
        resultado['codigo_corregido'] = codigo
        
        # Actualizar estadísticas
        if resultado['errores']:
            self.config.estadisticas['celdas_con_errores'] += 1
            self.config.estadisticas['errores_sintaxis'] += sum(1 for e in resultado['errores'] if e.get('tipo') == 'SyntaxError')
            self.config.estadisticas['errores_importacion'] += sum(1 for e in resultado['errores'] if 'import' in e.get('tipo', '').lower())
            self.config.estadisticas['errores_logica'] += sum(1 for e in resultado['errores'] if e.get('tipo') not in ['SyntaxError', 'ImportError'])
        
        self.config.estadisticas['advertencias'] += len(resultado['advertencias'])
        self.config.estadisticas['mejoras_aplicadas'] += len(resultado['correcciones'])
        
        # Log resumen de la celda
        self._log_resumen_celda(idx, resultado)
        
        return resultado
    
    def _intentar_corregir_sintaxis(self, codigo: str) -> str:
        """Intenta corregir errores de sintaxis comunes"""
        codigo_corregido = codigo
        
        # Correcciones comunes
        correcciones = [
            # Agregar : faltantes en estructuras de control
            (r'^\s*(if|elif|else|for|while|def|class|try|except|finally|with)\s+[^:]+$', r'\g<0>:', re.MULTILINE),
            # Corregir indentación inconsistente (tabs a espacios)
            (r'\t', '    ', 0),
            # Eliminar espacios al final de líneas
            (r' +$', '', re.MULTILINE),
            # Corregir comillas no cerradas simples
            (r"'([^']*?)$", r"'\1'", re.MULTILINE),
            # Corregir comillas no cerradas dobles
            (r'"([^"]*?)$', r'"\1"', re.MULTILINE),
            # Agregar paréntesis faltantes en print (Python 3)
            (r'print\s+([^(].*?)$', r'print(\1)', re.MULTILINE),
        ]
        
        for patron, reemplazo, flags in correcciones:
            if flags == 0:
                codigo_corregido = codigo_corregido.replace(patron, reemplazo)
            else:
                codigo_corregido = re.sub(patron, reemplazo, codigo_corregido, flags=flags)
        
        return codigo_corregido
    
    def _revisar_celda_markdown(self, celda, idx: int) -> Dict:
        """Revisa una celda de markdown"""
        contenido = celda.source
        resultado = {
            'indice': idx,
            'tipo_celda': 'markdown',
            'contenido_original': contenido,
            'errores': [],
            'advertencias': [],
            'correcciones': []
        }
        
        # Verificar markdown vacío
        if not contenido.strip():
            resultado['advertencias'].append({
                'tipo': 'Markdown vacío',
                'mensaje': 'La celda markdown está vacía',
                'severidad': 'BAJO'
            })
            return resultado
        
        # Verificar estructura de markdown
        contenido_corregido = contenido
        
        # Verificar headers mal formateados
        lineas = contenido.split('\n')
        lineas_corregidas = []
        
        for i, linea in enumerate(lineas):
            linea_corregida = linea
            
            # Corregir headers sin espacio después de #
            if re.match(r'^#{1,6}[^#\s]', linea):
                linea_corregida = re.sub(r'^(#{1,6})([^#\s])', r'\1 \2', linea)
                resultado['correcciones'].append({
                    'tipo': 'Formato header',
                    'linea': i + 1,
                    'descripcion': 'Agregado espacio después de #'
                })
            
            # Detectar líneas muy largas
            if len(linea) > 100:
                resultado['advertencias'].append({
                    'tipo': 'Línea larga',
                    'linea': i + 1,
                    'longitud': len(linea),
                    'severidad': 'BAJO'
                })
            
            lineas_corregidas.append(linea_corregida)
        
        contenido_corregido = '\n'.join(lineas_corregidas)
        
        # Verificar enlaces rotos (formato básico) - LÍNEA CORREGIDA
        # Línea 944 - CORREGIDA
        enlaces = re.findall(r"""
        $$
        ([^
        $$]+)\]$([^)]+)$""", contenido)
        for texto, url in enlaces:
            if url.startswith('#') or url.startswith('http'):
                continue
            if not os.path.exists(os.path.join(str(self.config.ruta_base), url)):
                resultado['advertencias'].append({
                    'tipo': 'Posible enlace roto',
                    'texto': texto,
                    'url': url,
                    'severidad': 'MEDIO'
                })
        
        resultado['contenido_corregido'] = contenido_corregido
        return resultado
    
    def _crear_celda_corregida(self, celda_original, resultado_revision: Dict):
        """Crea una celda corregida basada en los resultados de la revisión"""
        codigo_corregido = resultado_revision.get('codigo_corregido', celda_original.source)
        
        # Crear nueva celda
        celda_nueva = nbformat.v4.new_code_cell(source=codigo_corregido)
        
        # Agregar comentarios sobre las correcciones realizadas
        if resultado_revision['correcciones']:
            comentarios = ["# CORRECCIONES APLICADAS:"]
            for correccion in resultado_revision['correcciones']:
                comentarios.append(f"# - {correccion['tipo']}: {correccion['descripcion']}")
            comentarios.append("")
            
            # Agregar comentarios al inicio del código
            celda_nueva.source = '\n'.join(comentarios) + '\n' + codigo_corregido
        
        # Agregar metadata con información de la revisión
        celda_nueva.metadata['revision'] = {
            'fecha': datetime.now().isoformat(),
            'errores_encontrados': len(resultado_revision['errores']),
            'advertencias': len(resultado_revision['advertencias']),
            'correcciones_aplicadas': len(resultado_revision['correcciones'])
        }
        
        return celda_nueva
    
    def _log_resumen_celda(self, idx: int, resultado: Dict):
        """Registra un resumen de la revisión de una celda"""
        num_errores = len(resultado['errores'])
        num_advertencias = len(resultado['advertencias'])
        num_correcciones = len(resultado['correcciones'])
        
        if num_errores > 0:
            self.logger.warning(f"  ✗ Celda {idx + 1}: {num_errores} errores encontrados")
            for error in resultado['errores']:
                self.logger.error(f"    - {error['tipo']}: {error['mensaje']}")
        elif num_advertencias > 0:
            self.logger.info(f"  ⚠ Celda {idx + 1}: {num_advertencias} advertencias")
        else:
            self.logger.info(f"  ✓ Celda {idx + 1}: Sin problemas detectados")
        
        if num_correcciones > 0:
            self.logger.info(f"  ⚡ {num_correcciones} correcciones aplicadas")

# Celda 6: Generador de reportes
class GeneradorReportes:
    """Genera reportes detallados de la revisión"""
    
    def __init__(self, config: ConfiguracionRevisor, resultados: List[Dict]):
        self.config = config
        self.resultados = resultados
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
    def generar_reporte_completo(self):
        """Genera un reporte completo en múltiples formatos"""
        self.logger = self.config.logger
        
        self.logger.info("\nGenerando reportes...")
        
        # Generar reporte en texto
        self._generar_reporte_texto()
        
        # Generar reporte en HTML
        self._generar_reporte_html()
        
        # Generar reporte en JSON
        self._generar_reporte_json()
        
        # Generar resumen ejecutivo
        self._generar_resumen_ejecutivo()
        
    def _generar_reporte_texto(self):
        """Genera reporte en formato texto"""
        ruta_reporte = self.config.dir_reportes / f"reporte_{self.timestamp}.txt"
        
        with open(ruta_reporte, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"REPORTE DE REVISIÓN DE CÓDIGO\n")
            f.write(f"Archivo: {self.config.archivo_objetivo}\n")
            f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*80 + "\n\n")
            
            # Estadísticas generales
            f.write("ESTADÍSTICAS GENERALES:\n")
            f.write("-"*40 + "\n")
            for key, value in self.config.estadisticas.items():
                f.write(f"{key}: {value}\n")
            f.write("\n")
            
            # Detalle por celda
            f.write("DETALLE POR CELDA:\n")
            f.write("="*80 + "\n")
            
            for resultado in self.resultados:
                f.write(f"\nCELDA {resultado['indice'] + 1} ({resultado['tipo_celda']}):\n")
                f.write("-"*60 + "\n")
                
                # Errores
                if resultado['errores']:
                    f.write(f"ERRORES ({len(resultado['errores'])}):\n")
                    for error in resultado['errores']:
                        f.write(f"  • [{error.get('severidad', 'MEDIO')}] {error['tipo']}: {error['mensaje']}\n")
                        if 'sugerencia' in error:
                            f.write(f"    → Sugerencia: {error['sugerencia']}\n")
                
                # Advertencias
                if resultado['advertencias']:
                    f.write(f"\nADVERTENCIAS ({len(resultado['advertencias'])}):\n")
                    for adv in resultado['advertencias']:
                        f.write(f"  • [{adv.get('severidad', 'BAJO')}] {adv['tipo']}: {adv.get('mensaje', '')}\n")
                
                # Correcciones aplicadas
                if resultado.get('correcciones'):
                    f.write(f"\nCORRECCIONES APLICADAS ({len(resultado['correcciones'])}):\n")
                    for corr in resultado['correcciones']:
                        f.write(f"  • {corr['tipo']}: {corr['descripcion']}\n")
                
                # Métricas
                if resultado.get('metricas'):
                    f.write(f"\nMÉTRICAS:\n")
                    for metrica, valor in resultado['metricas'].items():
                        if isinstance(valor, dict):
                            f.write(f"  • {metrica}:\n")
                            for k, v in valor.items():
                                f.write(f"    - {k}: {v}\n")
                        else:
                            f.write(f"  • {metrica}: {valor}\n")
                
                f.write("\n")
        
        self.logger.info(f"Reporte de texto generado: {ruta_reporte}")
    
    def _generar_reporte_html(self):
        """Genera reporte en formato HTML"""
        ruta_reporte = self.config.dir_reportes / f"reporte_{self.timestamp}.html"
        
        html_content = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Reporte de Revisión - {self.config.archivo_objetivo}</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f4f4f4;
        }}
        .header {{
            background-color: #2c3e50;
            color: white;
            padding: 20px;
            border-radius: 5px;
            margin-bottom: 20px;
        }}
        .estadisticas {{
            background-color: white;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }}
        .celda {{
            background-color: white;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            margin-bottom: 15px;
        }}
        .error {{
            background-color: #ffebee;
            border-left: 4px solid #f44336;
            padding: 10px;
            margin: 5px 0;
        }}
        .advertencia {{
            background-color: #fff3e0;
            border-left: 4px solid #ff9800;
            padding: 10px;
            margin: 5px 0;
        }}
        .correccion {{
            background-color: #e8f5e9;
            border-left: 4px solid #4caf50;
            padding: 10px;
            margin: 5px 0;
        }}
        .severidad-CRITICO {{ color: #d32f2f; font-weight: bold; }}
        .severidad-ALTO {{ color: #f44336; font-weight: bold; }}
        .severidad-MEDIO {{ color: #ff9800; }}
        .severidad-BAJO {{ color: #ffc107; }}
        .codigo {{
            background-color: #f5f5f5;
            border: 1px solid #ddd;
            border-radius: 3px;
            padding: 10px;
            font-family: 'Courier New', monospace;
            overflow-x: auto;
        }}
        .metricas {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 10px;
            margin-top: 10px;
        }}
        .metrica-item {{
            background-color: #e3f2fd;
            padding: 10px;
            border-radius: 3px;
            text-align: center;
        }}
        .metrica-valor {{
            font-size: 24px;
            font-weight: bold;
            color: #1976d2;
        }}
        .resumen {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
            gap: 15px;
            margin: 20px 0;
        }}
        .resumen-item {{
            background-color: white;
            padding: 15px;
            border-radius: 5px;
            text-align: center;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }}
        .grafico {{
            margin: 20px 0;
        }}
        pre {{
            white-space: pre-wrap;
            word-wrap: break-word;
        }}
    </style>
</head>
<body>
    <div class="header">
        <h1>Reporte de Revisión de Código</h1>
        <p><strong>Archivo:</strong> {self.config.archivo_objetivo}</p>
        <p><strong>Fecha:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        <p><strong>Tiempo de análisis:</strong> {self.config.estadisticas['tiempo_ejecucion']:.2f} segundos</p>
    </div>
    
    <div class="estadisticas">
        <h2>Resumen Ejecutivo</h2>
        <div class="resumen">
            <div class="resumen-item">
                <div class="metrica-valor">{self.config.estadisticas['total_celdas']}</div>
                <div>Total Celdas</div>
            </div>
            <div class="resumen-item">
                <div class="metrica-valor">{self.config.estadisticas['celdas_con_errores']}</div>
                <div>Celdas con Errores</div>
            </div>
            <div class="resumen-item">
                <div class="metrica-valor">{self.config.estadisticas['errores_sintaxis']}</div>
                <div>Errores de Sintaxis</div>
            </div>
            <div class="resumen-item">
                <div class="metrica-valor">{self.config.estadisticas['advertencias']}</div>
                <div>Advertencias</div>
            </div>
            <div class="resumen-item">
                <div class="metrica-valor">{self.config.estadisticas['mejoras_aplicadas']}</div>
                <div>Mejoras Aplicadas</div>
            </div>
        </div>
    </div>
"""
        
        # Agregar detalles por celda
        html_content += '<div class="detalles">\n<h2>Detalle por Celda</h2>\n'
        
        for resultado in self.resultados:
            html_content += f'''
    <div class="celda">
        <h3>Celda {resultado['indice'] + 1} ({resultado['tipo_celda']})</h3>
'''
            
            # Errores
            if resultado['errores']:
                html_content += f'<h4>Errores ({len(resultado["errores"])})</h4>\n'
                for error in resultado['errores']:
                    severidad = error.get('severidad', 'MEDIO')
                    html_content += f'''
        <div class="error">
            <span class="severidad-{severidad}">[{severidad}]</span> 
            <strong>{error['tipo']}:</strong> {error['mensaje']}
'''
                    if 'sugerencia' in error:
                        html_content += f'<br><em>→ Sugerencia: {error["sugerencia"]}</em>'
                    html_content += '</div>\n'
            
            # Advertencias
            if resultado['advertencias']:
                html_content += f'<h4>Advertencias ({len(resultado["advertencias"])})</h4>\n'
                for adv in resultado['advertencias']:
                    severidad = adv.get('severidad', 'BAJO')
                    html_content += f'''
        <div class="advertencia">
            <span class="severidad-{severidad}">[{severidad}]</span> 
            <strong>{adv['tipo']}:</strong> {adv.get('mensaje', '')}
        </div>
'''
            
            # Correcciones
            if resultado.get('correcciones'):
                html_content += f'<h4>Correcciones Aplicadas ({len(resultado["correcciones"])})</h4>\n'
                for corr in resultado['correcciones']:
                    html_content += f'''
        <div class="correccion">
            <strong>{corr['tipo']}:</strong> {corr['descripcion']}
        </div>
'''
            
            # Métricas
            if resultado.get('metricas'):
                html_content += '<h4>Métricas</h4>\n<div class="metricas">\n'
                for metrica, valor in resultado['metricas'].items():
                     if isinstance(valor, dict):
                        for k, v in valor.items():
                            html_content += f'''
            <div class="metrica-item">
                <div class="metrica-valor">{v}</div>
                <div>{metrica} - {k}</div>
            </div>
'''
                else:
                        html_content += f'''
            <div class="metrica-item">
                <div class="metrica-valor">{valor}</div>
                <div>{metrica}</div>
            </div>
'''
                html_content += '</div>\n'
            
            html_content += '</div>\n'
        
        html_content += '</div>\n'
        
        # Cerrar HTML
        html_content += '''
</body>
</html>
'''
        
        # Guardar archivo
        with open(ruta_reporte, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        self.logger.info(f"Reporte HTML generado: {ruta_reporte}")
    
    def _generar_reporte_json(self):
        """Genera reporte en formato JSON"""
        ruta_reporte = self.config.dir_reportes / f"reporte_{self.timestamp}.json"
        
        reporte_json = {
            'metadata': {
                'archivo': self.config.archivo_objetivo,
                'fecha_revision': datetime.now().isoformat(),
                'version_revisor': '1.0.0',
                'ruta_completa': str(self.config.ruta_completa)
            },
            'estadisticas': self.config.estadisticas,
            'resultados_por_celda': self.resultados,
            'resumen': {
                'total_errores': sum(len(r['errores']) for r in self.resultados),
                'total_advertencias': sum(len(r['advertencias']) for r in self.resultados),
                'total_correcciones': sum(len(r.get('correcciones', [])) for r in self.resultados),
                'celdas_perfectas': sum(1 for r in self.resultados if not r['errores'] and not r['advertencias']),
                'severidad_maxima': self._calcular_severidad_maxima()
            }
        }
        
        with open(ruta_reporte, 'w', encoding='utf-8') as f:
            json.dump(reporte_json, f, indent=2, ensure_ascii=False, default=str)
        
        self.logger.info(f"Reporte JSON generado: {ruta_reporte}")
    
    def _generar_resumen_ejecutivo(self):
        """Genera un resumen ejecutivo conciso"""
        ruta_resumen = self.config.dir_reportes / f"resumen_ejecutivo_{self.timestamp}.md"
        
        total_errores = sum(len(r['errores']) for r in self.resultados)
        total_advertencias = sum(len(r['advertencias']) for r in self.resultados)
        total_correcciones = sum(len(r.get('correcciones', [])) for r in self.resultados)
        
        with open(ruta_resumen, 'w', encoding='utf-8') as f:
            f.write(f"# Resumen Ejecutivo - Revisión de Código\n\n")
            f.write(f"**Archivo:** `{self.config.archivo_objetivo}`\n")
            f.write(f"**Fecha:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            f.write("## Resultados Generales\n\n")
            
            # Estado general
            if total_errores == 0:
                f.write("✅ **Estado:** Sin errores críticos\n\n")
            else:
                f.write("❌ **Estado:** Se encontraron errores que requieren atención\n\n")
            
            # Tabla de estadísticas
            f.write("| Métrica | Valor |\n")
            f.write("|---------|-------|\n")
            f.write(f"| Total de celdas | {self.config.estadisticas['total_celdas']} |\n")
            f.write(f"| Celdas con errores | {self.config.estadisticas['celdas_con_errores']} |\n")
            f.write(f"| Total errores | {total_errores} |\n")
            f.write(f"| Total advertencias | {total_advertencias} |\n")
            f.write(f"| Correcciones aplicadas | {total_correcciones} |\n")
            f.write(f"| Tiempo de análisis | {self.config.estadisticas['tiempo_ejecucion']:.2f}s |\n\n")
            
            # Recomendaciones principales
            f.write("## Recomendaciones Principales\n\n")
            recomendaciones = self._generar_recomendaciones()
            for i, rec in enumerate(recomendaciones, 1):
                f.write(f"{i}. {rec}\n")
            
        self.logger.info(f"Resumen ejecutivo generado: {ruta_resumen}")
    
    def _calcular_severidad_maxima(self):
        """Calcula la severidad máxima encontrada en todos los errores"""
        severidades = ['BAJO', 'MEDIO', 'ALTO', 'CRITICO']
        max_severidad = 'BAJO'
        
        for resultado in self.resultados:
            for error in resultado['errores']:
                severidad = error.get('severidad', 'MEDIO')
                if severidades.index(severidad) > severidades.index(max_severidad):
                    max_severidad = severidad
                    
            for advertencia in resultado['advertencias']:
                severidad = advertencia.get('severidad', 'BAJO')
                if severidades.index(severidad) > severidades.index(max_severidad):
                    max_severidad = severidad
                    
        return max_severidad
    
    def _generar_recomendaciones(self):
        """Genera recomendaciones basadas en los resultados del análisis"""
        recomendaciones = []
        
        # Analizar patrones de errores
        errores_por_tipo = defaultdict(int)
        for resultado in self.resultados:
            for error in resultado['errores']:
                errores_por_tipo[error['tipo']] += 1
        
        # Recomendaciones basadas en errores
        if errores_por_tipo.get('SyntaxError', 0) > 0:
            recomendaciones.append("Revisar y corregir todos los errores de sintaxis antes de ejecutar el código")
        
        if errores_por_tipo.get('ImportError', 0) > 0:
            recomendaciones.append("Verificar que todos los módulos necesarios estén instalados e importados correctamente")
        
        if errores_por_tipo.get('Variable no definida', 0) > 0:
            recomendaciones.append("Asegurar que todas las variables se definan antes de usarlas")
        
        # Recomendaciones basadas en estadísticas
        porcentaje_errores = (self.config.estadisticas['celdas_con_errores'] / 
                             self.config.estadisticas['total_celdas'] * 100) if self.config.estadisticas['total_celdas'] > 0 else 0
        
        if porcentaje_errores > 50:
            recomendaciones.append("Considerar una revisión completa del código, ya que más del 50% de las celdas tienen errores")
        
        if self.config.estadisticas['advertencias'] > 20:
            recomendaciones.append("Revisar las advertencias de estilo para mejorar la legibilidad del código")
        
        # Si no hay problemas significativos
        if not recomendaciones:
            recomendaciones.append("El código está en buen estado. Mantener las buenas prácticas actuales")
        
        return recomendaciones

# Celda 7: Función principal de ejecución
def ejecutar_revision_completa():
    """Función principal que ejecuta todo el proceso de revisión"""
    print("="*80)
    print("DETECTOR DE ERRORES - REVISIÓN COMPLETA DE CÓDIGO")
    print("="*80)
    
    try:
        # Crear configuración
        config = ConfiguracionRevisor()
        
        # Crear revisor
        revisor = RevisorNotebook(config)
        
        # Cargar notebook
        if not revisor.cargar_notebook():
            print("ERROR: No se pudo cargar el notebook")
            return False
        
        # Ejecutar revisión
        revisor.revisar_notebook()
        
        # Generar reportes
        generador = GeneradorReportes(config, revisor.resultados_totales)
        generador.generar_reporte_completo()
        
        # Guardar notebook corregido
        if revisor.notebook_corregido:
            ruta_corregido = config.dir_corregidos / f"{config.archivo_objetivo.replace('.ipynb', '')}_corregido_{datetime.now().strftime('%Y%m%d_%H%M%S')}.ipynb"
            
            with open(ruta_corregido, 'w', encoding='utf-8') as f:
                nbformat.write(revisor.notebook_corregido, f)
            
            print(f"\n✓ Notebook corregido guardado en: {ruta_corregido}")
        
        # Mostrar resumen final
        print("\n" + "="*80)
        print("RESUMEN FINAL")
        print("="*80)
        print(f"Total de celdas analizadas: {config.estadisticas['total_celdas']}")
        print(f"Celdas con errores: {config.estadisticas['celdas_con_errores']}")
        print(f"Total de errores encontrados: {sum(len(r['errores']) for r in revisor.resultados_totales)}")
        print(f"Total de advertencias: {config.estadisticas['advertencias']}")
        print(f"Correcciones aplicadas: {config.estadisticas['mejoras_aplicadas']}")
        print(f"Tiempo de ejecución: {config.estadisticas['tiempo_ejecucion']:.2f} segundos")
        print("="*80)
        
        # Verificar si es necesario revisar el propio detector
        if config.archivo_objetivo == "Detector_errores.ipynb":
            print("\n⚠️  NOTA: Se ha revisado el propio archivo detector.")
            print("    Considere aplicar las correcciones sugeridas para mejorar este revisor.")
        
        return True
        
    except Exception as e:
        print(f"\n❌ ERROR CRÍTICO: {str(e)}")
        print(traceback.format_exc())
        return False

# Celda 8: Auto-revisión del detector
class AutoRevisorDetector:
    """Clase especial para auto-revisar el código del detector"""
    
    def __init__(self):
        self.problemas_detectados = []
        self.mejoras_sugeridas = []
        
    def auto_revisar(self):
        """Realiza una auto-revisión del código del detector"""
        print("\n" + "="*80)
        print("AUTO-REVISIÓN DEL DETECTOR DE ERRORES")
        print("="*80)
        
        # Verificar estructura del código
        self._verificar_estructura()
        
        # Verificar dependencias
        self._verificar_dependencias()
        
        # Verificar configuración
        self._verificar_configuracion()
        
        # Mostrar resultados
        self._mostrar_resultados_autorevision()
    
    def _verificar_estructura(self):
        """Verifica la estructura del código del detector"""
        clases_requeridas = [
            'ConfiguracionRevisor',
            'AnalizadorCodigo', 
            'RevisorNotebook',
            'GeneradorReportes',
            'AnalizadorAST'
        ]
        
        # Verificar que todas las clases estén definidas
        for clase in clases_requeridas:
            if clase not in globals():
                self.problemas_detectados.append(f"Clase '{clase}' no está definida")
            else:
                self.mejoras_sugeridas.append(f"✓ Clase '{clase}' definida correctamente")
    
    def _verificar_dependencias(self):
        """Verifica que todas las dependencias estén disponibles"""
        dependencias = {
            'nbformat': 'Procesamiento de notebooks',
            'ast': 'Análisis de sintaxis',
            'pycodestyle': 'Verificación de estilo PEP8',
            'autopep8': 'Corrección automática de estilo'
        }
        
        for modulo, descripcion in dependencias.items():
            try:
                __import__(modulo)
                self.mejoras_sugeridas.append(f"✓ {modulo}: {descripcion}")
            except ImportError:
                self.problemas_detectados.append(f"Módulo '{modulo}' no disponible: {descripcion}")
    
    def _verificar_configuracion(self):
        """Verifica la configuración del sistema"""
        try:
            config = ConfiguracionRevisor()
            
            # Verificar rutas
            if not config.ruta_base.exists():
                self.problemas_detectados.append(f"Ruta base no existe: {config.ruta_base}")
            else:
                self.mejoras_sugeridas.append(f"✓ Ruta base configurada: {config.ruta_base}")
                
            # Verificar estructura de directorios
            for dir_name in ['dir_reportes', 'dir_corregidos', 'dir_logs', 'dir_backups']:
                if hasattr(config, dir_name):
                    dir_path = getattr(config, dir_name)
                    if dir_path.exists():
                        self.mejoras_sugeridas.append(f"✓ Directorio {dir_name} creado")
                        
        except Exception as e:
            self.problemas_detectados.append(f"Error al verificar configuración: {str(e)}")
    
    def _mostrar_resultados_autorevision(self):
        """Muestra los resultados de la auto-revisión"""
        print("\nPROBLEMAS DETECTADOS:")
        if self.problemas_detectados:
            for problema in self.problemas_detectados:
                print(f"  ❌ {problema}")
        else:
            print("  ✅ No se detectaron problemas")
        
        print("\nVERIFICACIONES EXITOSAS:")
        for mejora in self.mejoras_sugeridas:
            print(f"  {mejora}")
        
        print("\n" + "="*80)

# Celda 9: Utilidades adicionales y funciones auxiliares
class UtilidadesRevision:
    """Utilidades adicionales para la revisión de código"""
    
    @staticmethod
    def verificar_notebook_existe(ruta: Path) -> bool:
        """Verifica si un notebook existe en la ruta especificada"""
        return ruta.exists() and ruta.suffix == '.ipynb'
    
    @staticmethod
    def listar_notebooks_directorio(ruta_directorio: Path) -> List[Path]:
        """Lista todos los notebooks en un directorio"""
        return list(ruta_directorio.glob('*.ipynb'))
    
    @staticmethod
    def comparar_notebooks(notebook1: Path, notebook2: Path) -> Dict:
        """Compara dos notebooks y muestra las diferencias"""
        with open(notebook1, 'r', encoding='utf-8') as f1:
            nb1 = nbformat.read(f1, as_version=4)
        
        with open(notebook2, 'r', encoding='utf-8') as f2:
            nb2 = nbformat.read(f2, as_version=4)
        
        diferencias = {
            'num_celdas': len(nb1.cells) - len(nb2.cells),
            'celdas_diferentes': 0,
            'detalles': []
        }
        
        for i, (celda1, celda2) in enumerate(zip(nb1.cells, nb2.cells)):
            if celda1.source != celda2.source:
                diferencias['celdas_diferentes'] += 1
                diferencias['detalles'].append({
                    'celda': i + 1,
                    'tipo_cambio': 'contenido diferente'
                })
        
        return diferencias
    
    @staticmethod
    def extraer_imports_notebook(notebook_path: Path) -> List[str]:
        """Extrae todos los imports de un notebook"""
        imports = set()
        
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
        
        for celda in notebook.cells:
            if celda.cell_type == 'code':
                try:
                    tree = ast.parse(celda.source)
                    for node in ast.walk(tree):
                        if isinstance(node, ast.Import):
                            for alias in node.names:
                                imports.add(alias.name)
                        elif isinstance(node, ast.ImportFrom):
                            if node.module:
                                imports.add(node.module)
                except:
                    pass
        
        return sorted(list(imports))
    
    @staticmethod
    def generar_requirements(imports: List[str]) -> str:
        """Genera un archivo requirements.txt basado en los imports"""
        # Mapeo de imports comunes a nombres de paquetes pip
        mapeo_paquetes = {
            'sklearn': 'scikit-learn',
            'cv2': 'opencv-python',
            'PIL': 'Pillow',
            'bs4': 'beautifulsoup4',
            'yaml': 'pyyaml',
            'dotenv': 'python-dotenv'
        }
        
        requirements = []
        modulos_stdlib = set(sys.stdlib_module_names) if hasattr(sys, 'stdlib_module_names') else set()
        
        for imp in imports:
            # Obtener el módulo principal
            modulo_principal = imp.split('.')[0]
            
            # Saltar módulos de la librería estándar
            if modulo_principal in modulos_stdlib:
                continue
            
            # Usar mapeo si existe
            if modulo_principal in mapeo_paquetes:
                requirements.append(mapeo_paquetes[modulo_principal])
            else:
                requirements.append(modulo_principal)
        
        return '\n'.join(sorted(set(requirements)))

# Celda 10: Ejecución principal y menú interactivo
def menu_principal():
    """Menú principal interactivo del detector de errores"""
    while True:
        print("\n" + "="*80)
        print("DETECTOR DE ERRORES - MENÚ PRINCIPAL")
        print("="*80)
        print("1. Revisar Detector_errores.ipynb")
        print("2. Revisar otro notebook")
        print("3. Revisar todos los notebooks del directorio")
        print("4. Auto-revisión del detector")
        print("5. Comparar notebooks")
        print("6. Extraer dependencias de un notebook")
        print("7. Ver estadísticas de revisiones anteriores")
        print("8. Salir")
        print("="*80)
        
        opcion = input("\nSeleccione una opción (1-8): ").strip()
        
        if opcion == '1':
            print("\nRevisando Detector_errores.ipynb...")
            ejecutar_revision_completa()
            
        elif opcion == '2':
            nombre = input("\nIngrese el nombre del notebook a revisar: ").strip()
            config = ConfiguracionRevisor()
            config.archivo_objetivo = nombre
            config.ruta_completa = config.ruta_base / nombre
            
            if not config.ruta_completa.exists():
                print(f"\n❌ Error: No se encontró el archivo {nombre}")
                continue
                
            revisor = RevisorNotebook(config)
            if revisor.cargar_notebook():
                revisor.revisar_notebook()
                generador = GeneradorReportes(config, revisor.resultados_totales)
                generador.generar_reporte_completo()
                print("\n✅ Revisión completada")
            
        elif opcion == '3':
            print("\nRevisando todos los notebooks del directorio...")
            config = ConfiguracionRevisor()
            notebooks = UtilidadesRevision.listar_notebooks_directorio(config.ruta_base)
            
            print(f"\nSe encontraron {len(notebooks)} notebooks")
            
            for nb_path in notebooks:
                print(f"\n--- Revisando: {nb_path.name} ---")
                config_nb = ConfiguracionRevisor()
                config_nb.archivo_objetivo = nb_path.name
                config_nb.ruta_completa = nb_path
                
                revisor = RevisorNotebook(config_nb)
                if revisor.cargar_notebook():
                    revisor.revisar_notebook()
                    generador = GeneradorReportes(config_nb, revisor.resultados_totales)
                    generador.generar_reporte_completo()
            
            print("\n✅ Revisión masiva completada")
            
        elif opcion == '4':
            print("\nEjecutando auto-revisión...")
            auto_revisor = AutoRevisorDetector()
            auto_revisor.auto_revisar()
            
        elif opcion == '5':
            nb1 = input("\nIngrese el nombre del primer notebook: ").strip()
            nb2 = input("Ingrese el nombre del segundo notebook: ").strip()
            
            config = ConfiguracionRevisor()
            path1 = config.ruta_base / nb1
            path2 = config.ruta_base / nb2
            
            if not path1.exists() or not path2.exists():
                print("\n❌ Error: Uno o ambos archivos no existen")
                continue
                
            diferencias = UtilidadesRevision.comparar_notebooks(path1, path2)
            
            print(f"\n--- Comparación: {nb1} vs {nb2} ---")
            print(f"Diferencia en número de celdas: {diferencias['num_celdas']}")
            print(f"Celdas con contenido diferente: {diferencias['celdas_diferentes']}")
            
            if diferencias['detalles']:
                print("\nDetalles de diferencias:")
                for detalle in diferencias['detalles'][:10]:  # Mostrar máximo 10
                    print(f"  - Celda {detalle['celda']}: {detalle['tipo_cambio']}")
                    
        elif opcion == '6':
            nombre = input("\nIngrese el nombre del notebook: ").strip()
            config = ConfiguracionRevisor()
            nb_path = config.ruta_base / nombre
            
            if not nb_path.exists():
                print(f"\n❌ Error: No se encontró el archivo {nombre}")
                continue
                
            imports = UtilidadesRevision.extraer_imports_notebook(nb_path)
            requirements = UtilidadesRevision.generar_requirements(imports)
            
            print(f"\n--- Dependencias encontradas en {nombre} ---")
            print("\nImports detectados:")
            for imp in imports:
                print(f"  - {imp}")
            
            print("\n--- Requirements.txt sugerido ---")
            print(requirements)
            
            # Opción de guardar
            guardar = input("\n¿Desea guardar requirements.txt? (s/n): ").strip().lower()
            if guardar == 's':
                req_path = config.ruta_base / 'requirements_generado.txt'
                with open(req_path, 'w') as f:
                    f.write(requirements)
                print(f"✅ Guardado en: {req_path}")
                
        elif opcion == '7':
            print("\n--- Estadísticas de Revisiones Anteriores ---")
            config = ConfiguracionRevisor()
            
            # Buscar todos los reportes JSON
            reportes_json = list(config.dir_reportes.glob('reporte_*.json'))
            
            if not reportes_json:
                print("No se encontraron revisiones anteriores")
                continue
                
            estadisticas_totales = {
                'total_revisiones': len(reportes_json),
                'total_errores_historico': 0,
                'total_advertencias_historico': 0,
                'archivos_revisados': set()
            }
            
            for reporte_path in reportes_json:
                with open(reporte_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    
                estadisticas_totales['total_errores_historico'] += data['resumen']['total_errores']
                estadisticas_totales['total_advertencias_historico'] += data['resumen']['total_advertencias']
                estadisticas_totales['archivos_revisados'].add(data['metadata']['archivo'])
            
            print(f"\nTotal de revisiones realizadas: {estadisticas_totales['total_revisiones']}")
            print(f"Archivos únicos revisados: {len(estadisticas_totales['archivos_revisados'])}")
            print(f"Total histórico de errores: {estadisticas_totales['total_errores_historico']}")
            print(f"Total histórico de advertencias: {estadisticas_totales['total_advertencias_historico']}")
            
            # Mostrar últimas 5 revisiones
            print("\n--- Últimas 5 revisiones ---")
            reportes_recientes = sorted(reportes_json, key=lambda x: x.stat().st_mtime, reverse=True)[:5]
            
            for reporte in reportes_recientes:
                with open(reporte, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                fecha = datetime.fromisoformat(data['metadata']['fecha_revision'])
                print(f"\n  • {data['metadata']['archivo']} - {fecha.strftime('%Y-%m-%d %H:%M')}")
                print(f"    Errores: {data['resumen']['total_errores']} | Advertencias: {data['resumen']['total_advertencias']}")
                
        elif opcion == '8':
            print("\n¡Hasta luego! 👋")
            break
            
        else:
            print("\n❌ Opción no válida. Por favor, seleccione una opción del 1 al 8.")
        
        input("\nPresione Enter para continuar...")

# Celda 11: Punto de entrada principal
if __name__ == "__main__":
    """Punto de entrada principal del programa"""
    
    # Verificar si se ejecuta directamente o desde Jupyter
    try:
        get_ipython()
        # Estamos en Jupyter
        print("🔍 Detector de Errores - Modo Jupyter")
        print("\nOpciones disponibles:")
        print("1. Ejecutar revisión completa: ejecutar_revision_completa()")
        print("2. Auto-revisión: AutoRevisorDetector().auto_revisar()")
        print("3. Menú interactivo: menu_principal()")
        print("\nPara comenzar, ejecute una de las funciones anteriores.")
        
    except NameError:
        # Ejecución normal de Python
        print("🔍 Detector de Errores - Versión 1.0")
        print("Desarrollado para revisar y optimizar código en Jupyter Notebooks")
        
        # Verificar argumentos de línea de comandos
        if len(sys.argv) > 1:
            if sys.argv[1] == '--auto':
                # Modo automático: revisar Detector_errores.ipynb
                ejecutar_revision_completa()
            elif sys.argv[1] == '--menu':
                # Modo menú
                menu_principal()
            elif sys.argv[1] == '--help':
                print("\nUso:")
                print("  python detector_errores.py          # Ejecuta revisión de Detector_errores.ipynb")
                print("  python detector_errores.py --auto   # Modo automático")
                print("  python detector_errores.py --menu   # Menú interactivo")
                print("  python detector_errores.py --help   # Muestra esta ayuda")
            else:
                # Intentar revisar el archivo especificado
                config = ConfiguracionRevisor()
                config.archivo_objetivo = sys.argv[1]
                config.ruta_completa = config.ruta_base / sys.argv[1]
                
                revisor = RevisorNotebook(config)
                if revisor.cargar_notebook():
                    revisor.revisar_notebook()
                    generador = GeneradorReportes(config, revisor.resultados_totales)
                    generador.generar_reporte_completo()
        else:
            # Sin argumentos: ejecutar revisión por defecto
            ejecutar_revision_completa()

🔍 Detector de Errores - Modo Jupyter

Opciones disponibles:
1. Ejecutar revisión completa: ejecutar_revision_completa()
2. Auto-revisión: AutoRevisorDetector().auto_revisar()
3. Menú interactivo: menu_principal()

Para comenzar, ejecute una de las funciones anteriores.


In [2]:
ejecutar_revision_completa()

DETECTOR DE ERRORES - REVISIÓN COMPLETA DE CÓDIGO
2025-08-20 13:11:07 - RevisorCodigo - INFO - ================================================================================
2025-08-20 13:11:07 - RevisorCodigo - INFO - Iniciando revisión de: Detector_errores.ipynb
2025-08-20 13:11:07 - RevisorCodigo - INFO - Ruta completa: C:\Users\Alicia_Piero\Documents\Repo_AIEP\MIP_QUILLOTA\Proyecto_METGO_3D\Detector_errores.ipynb
2025-08-20 13:11:07 - RevisorCodigo - INFO - ================================================================================
2025-08-20 13:11:07 - RevisorCodigo - INFO - Cargando notebook: C:\Users\Alicia_Piero\Documents\Repo_AIEP\MIP_QUILLOTA\Proyecto_METGO_3D\Detector_errores.ipynb
2025-08-20 13:11:07 - RevisorCodigo - INFO - Notebook cargado exitosamente. Total de celdas: 10
2025-08-20 13:11:07 - RevisorCodigo - INFO - Backup creado: C:\Users\Alicia_Piero\Documents\Repo_AIEP\MIP_QUILLOTA\Proyecto_METGO_3D\backups\Detector_errores.ipynb.backup_20250820_131107
2025-08-

True